In [3]:
import pandas as pd 
import numpy as np 
import re
import requests
from bs4 import BeautifulSoup
import geocoder

In [24]:
url_to_open = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url_to_open)

soup = BeautifulSoup(html.content, 'lxml')
table = soup.find("table").find("tbody").find_all("tr")

postalCodes_list = []
borough_list = []
neighbourhood_list = []

# chars to remove
bad_chars = '()'
i = 0
for row in table:
    rows = row.find_all('td')
    for x in rows:
        x_clean = x.get_text().strip()
        match = re.search(r"\([^)]+\)", x_clean)
        if match:
            match_text = match.group(0)
            for c in bad_chars: match_text = match_text.replace(c, "")
            match_text = match_text.replace('/', ",")
            neighbourhood_list.append(match_text)
            postalCodes_list.append(x_clean[:3])
            borough = (x_clean[3:]).split('(')[0]
            borough_list.append(borough)

#postalCodes = {"Postal Code" : postalCodes_list}
#boroughs = {"Boroughs" : borough_list}
#neighbourhoods = {"Neighbourhoods" : neighbourhood_list}

In [30]:
column_names= ['Postal Codes', 'Boroughs', 'Neighbourhoods', 'Latitude', 'Logitude']
df  = pd.DataFrame(columns=column_names)
df
# df = pd.concat([df1, df2, df3], axis=1, sort=False)
#df = df[~df.Boroughs.str.contains('Not assigned')].reset_index(drop=True)

,Postal Codes,Boroughs,Neighbourhoods,Latitude,Logitude


In [29]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
for x in range(len(postalCodes_list)):
    print(postalCodes_list[x])
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalCodes_list[x]))
    print(g)
    lat_lng_coords = g.latlng
    print(lat_lng_coords)

    p = postalCodes_list[x]
    b = borough_list[x]
    n = neighbourhood_list[x]
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    df = df.append({'Postal Codes' : p,
                    'Boroughs' : b,
                    'Neighbourhoods' : n,
                    'Latitude' : latitude,
                    'Logitude' : longitude},
                    ignore_index=True
    )
df


M3A
<[OK] Arcgis - Geocode [M3A]>
[43.75242000000003, -79.32924245299995]
M4A
<[OK] Arcgis - Geocode [M4A]>
[43.73060024600005, -79.31326499999994]
M5A
<[OK] Arcgis - Geocode [M5A]>
[43.65029500000003, -79.35916572299999]
M6A
<[OK] Arcgis - Geocode [M6A]>
[43.72327000000007, -79.45128601699997]
M9A
<[OK] Arcgis - Geocode [M9A]>
[43.66229908300005, -79.52819499999998]
M1B
<[OK] Arcgis - Geocode [M1B]>
[43.811525000000074, -79.19551746399998]
M3B
<[OK] Arcgis - Geocode [M3B]>
[43.749055000000055, -79.36222672499997]
M4B
<[OK] Arcgis - Geocode [M4B]>
[43.707535000000064, -79.31177329699995]
M5B
<[OK] Arcgis - Geocode [M5B]>
[43.65736301100003, -79.37817999999999]
M6B
<[OK] Arcgis - Geocode [M6B]>
[43.70799000000005, -79.44836733199998]
M9B
<[OK] Arcgis - Geocode [M9B]>
[43.65034698100004, -79.55503999999996]
M1C
<[OK] Arcgis - Geocode [M1C]>
[43.78566500000005, -79.15872457299997]
M3C
<[OK] Arcgis - Geocode [M3C]>
[43.72142500000007, -79.34345422799998]
M4C
<[OK] Arcgis - Geocode [M4C]>
[

,Postal Codes,Boroughs,Neighbourhoods,Latitude,Logitude
0,M3A,North York,Parkwoods,43.752420,-79.329242
1,M4A,North York,Victoria Village,43.730600,-79.313265
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.650295,-79.359166
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723270,-79.451286
4,M9A,Etobicoke,Islington Avenue,43.662299,-79.528195
...,...,...,...,...,...
97,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653760,-79.510890
98,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381302
99,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.648690,-79.385440
100,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.632835,-79.489550


In [31]:
df.shape

(0, 5)